# EXAM - HUMANITARIAN NGOS IN DK

**READ ME:**

This notebook is created for collecting tweets for our exam project with the purpose of analysing Danish humanitarians communication during Covid-19.

As the notebook contains code for scraping twitter, **do not run *restart and run all* cells** when editing the code!


In [1]:
#import libaries

import tweepy
import jsonpickle
from collections import defaultdict
import json
import nltk 
import pandas as pd
import re
import string
import time
import datetime
import regex
import re
import numpy as np

#NLTK 

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer # Porter is used below. This is an alternative, harsher stemmer. 
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet



[nltk_data] Downloading package stopwords to /Users/Sofie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Sofie/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Sofie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Sofie/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
#importing keys for twitter api
from AppCred import CONSUMER_KEY, CONSUMER_SECRET
from AppCred import ACCESS_TOKEN, ACCESS_TOKEN_SECRET

#authentication process to access the Twitter API 
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

#pass the auth variable to the API function
api = tweepy.API(auth, 
          wait_on_rate_limit=True,
          wait_on_rate_limit_notify=True)


# User information

In [3]:
#list of all actors 
userlist_12 = ["@PlanBornefonden",'@CARE_Danmark', '@DRC_dk','@oxfamibis','@msf_dk','@ActionAidDK',
               '@redbarnetdk','@danskrodekors','@SOS_Bornebyerne','@UNICEFDK','@noedhjaelp',
              '@amnestydk', '@BornsVilkar', '@WFP_DK', '@danmissiondk', '@blaakorsdanmark',
           '@menneskeret', '@RTC_DK', '@DignityDK', '@UNDP_Danmark', '@globaltfokus',
           '@RefWelcome', '@DKIndsamling', '@Feministisk_DK', '@92_gruppen', '@ADRA_Danmark',
           '@lgbt_asylum', '@MissionEast', '@AVestegnen', '@CaritasDanmark']


In [4]:
#same user data through api 
user_data_12 = {}

for user in userlist_12:
    data = api.get_user(user) 
    user_data_12[user] = data

In [5]:
#create dataframe for user information
actors_df = pd.DataFrame()

In [6]:
#save usernames to data frame

usernames = []
for key, value in user_data_12.items():
    usernames.append(key)

actors_df['username'] = usernames

In [7]:
#create lists for informations we want to store
status_n = []
followers_n = []
friends_n = []
location = []
bio_description = []
url = []

for key, value in user_data_12.items():
    
    #appending information to lists
    status_n.append(user_data_12[key]._json['statuses_count'])
    followers_n.append(user_data_12[key]._json['followers_count'])
    friends_n.append(user_data_12[key]._json['friends_count'])
    location.append(user_data_12[key]._json['location'])
    url.append(user_data_12[key]._json['url'])
    bio_description.append(user_data_12[key]._json['description'])
    
    print(key, ', number of status:', user_data_12[key]._json['statuses_count'], 
          'followers:', user_data_12[key]._json['followers_count'], 'friends:',
          user_data_12[key]._json['friends_count'])

@PlanBornefonden , number of status: 2656 followers: 2010 friends: 1473
@CARE_Danmark , number of status: 3017 followers: 1740 friends: 761
@DRC_dk , number of status: 3313 followers: 6717 friends: 586
@oxfamibis , number of status: 5192 followers: 4711 friends: 1238
@msf_dk , number of status: 5326 followers: 5510 friends: 506
@ActionAidDK , number of status: 4236 followers: 4803 friends: 1207
@redbarnetdk , number of status: 6070 followers: 7610 friends: 1425
@danskrodekors , number of status: 4343 followers: 9461 friends: 1419
@SOS_Bornebyerne , number of status: 38 followers: 270 friends: 70
@UNICEFDK , number of status: 3779 followers: 3802 friends: 1339
@noedhjaelp , number of status: 6799 followers: 8753 friends: 1621
@amnestydk , number of status: 14690 followers: 7245 friends: 1914
@BornsVilkar , number of status: 2000 followers: 4629 friends: 879
@WFP_DK , number of status: 10283 followers: 3648 friends: 1470
@danmissiondk , number of status: 1277 followers: 979 friends: 357


In [8]:
#Add lists to dataframe in new columns

actors_df['status_n'], actors_df['followers_n'], actors_df['friends_n'], actors_df['location'], actors_df['bio_description'], actors_df['url'] = [status_n, 
                                                  followers_n, friends_n, 
                                                  location, bio_description, url
                                                 ]

In [9]:
actors_df

,username,status_n,followers_n,friends_n,location,bio_description,url
0,@PlanBornefonden,2656,2010,1473,"Copenhagen, Denmark","Vi er en del af @Planglobal, som arbejder i fl...",https://t.co/Zh6MLSRJ75
1,@CARE_Danmark,3017,1740,761,"København, Danmark","Grøn udviklings- og nødhjælpsorganisation, der...",https://t.co/1gXSbYZIA7
2,@DRC_dk,3313,6717,586,,Vi arbejder med at skabe varige løsninger for ...,https://t.co/PWmbi9nO5F
3,@oxfamibis,5192,4711,1238,"Copenhagen, Denmark",We are the Danish member of @oxfam - a global ...,https://t.co/LWhX3j8LVd
4,@msf_dk,5326,5510,506,"København, Danmark",Læger uden Grænser (MSF) er en int. humanitær ...,http://t.co/pHwNcEWbHv
5,@ActionAidDK,4236,4803,1207,Copenhagen Denmark,We fight poverty by strenghtening human rights...,https://t.co/6j7pcAQXJs
6,@redbarnetdk,6070,7610,1425,Danmark,"Nyt fra Red Barnet - nyheder, holdninger og ba...",https://t.co/G9RiddTagU
7,@danskrodekors,4343,9461,1419,Denmark,Vi er verdens største humanitære organisation....,https://t.co/BSk8ZON7W0
8,@SOS_Bornebyerne,38,270,70,,Vi giver forældreløse og udsatte børn en barnd...,http://t.co/Z3MaqNfLDl
9,@UNICEFDK,3779,3802,1339,"Copenhagen, Denmark",Vi hjælper børn over hele verden med at overle...,https://t.co/XS4ixM72vk


In [11]:
#uncomment to store in excel 

#actors_df.to_excel('All_actors.xlsx')

# Scraping users tweets and retweets

In [4]:
#Actors for scraping (first iteration) - 12 NGOs from Danmarks Indsamlingen 

userlist = ["@PlanBornefonden",'@CARE_Danmark', '@DRC_dk','@oxfamibis','@msf_dk','@ActionAidDK','@redbarnetdk',
            '@danskrodekors','@SOS_Bornebyerne','@UNICEFDK','@noedhjaelp']


In [5]:
# DO NOT RUN WHEN WE HAVE COLLECTED SOME TWEETS !!!

user_collected = [] # list of users already collected


In [6]:
userlist_to_collect = [user for user in userlist if user not in user_collected]

In [7]:
userlist_to_collect

['@PlanBornefonden',
 '@CARE_Danmark',
 '@DRC_dk',
 '@oxfamibis',
 '@msf_dk',
 '@ActionAidDK',
 '@redbarnetdk',
 '@danskrodekors',
 '@SOS_Bornebyerne',
 '@UNICEFDK',
 '@noedhjaelp']

In [8]:
#making a sleeper function 

def sleep_15_mins():
    print("Sleeping for 15 mins ...")
    for i in range(15):
        time.sleep(60)
        print(14 - i, "minutes of sleep remaining! ;pp")

## Loop for collecting tweets 

In [10]:

tweets = []
startDate = datetime.datetime(2021, 5, 15, 0, 0, 0)
endDate   = datetime.datetime(2022, 1, 1, 0, 0, 0)

all_tweets = {}

for user in userlist_to_collect:
    tweetCount = 0
    # Scraping status objects from the 'user' screen name
    user_tweets = []
    print("collecting tweets for:", user)
    c = tweepy.Cursor(api.user_timeline,
                      screen_name=user,
                      tweet_mode="extended" # to get the full text from a tweet.
                     ).items()
    while True:
        try:
            status = c.next()
            # check dato between start and end date, if so, save
            date = status.created_at
            if date < endDate and date >= startDate:

                #check for tweet or retweet
                retweet_match = re.search(r'^RT @\S+: ', status.full_text)
                if retweet_match:
                    tweet   = status.retweeted_status.full_text
                    retweet = retweet_match.group(0)
                else:
                    tweet   = status.full_text
                    retweet = None # use None to signal that this is not a retweet!
                user_tweets.append({"date": date, "tweet": tweet, "retweet": retweet})

                tweetCount += 1
        except tweepy.TweepError:
            sleep_15_mins()
            continue
        except StopIteration:
            break

    all_tweets[user] = user_tweets

    print('Done collecting {} tweets for user {}'.format(tweetCount, user))
        #user_collected.extend(user) # add the user to the list of user you have collected. 

    
#take either tweet or retweet extended 

collecting tweets for: @PlanBornefonden
Sleeping for 15 mins ...
14 minutes of sleep remaining! ;pp
13 minutes of sleep remaining! ;pp
12 minutes of sleep remaining! ;pp
11 minutes of sleep remaining! ;pp
10 minutes of sleep remaining! ;pp
9 minutes of sleep remaining! ;pp
8 minutes of sleep remaining! ;pp
7 minutes of sleep remaining! ;pp
6 minutes of sleep remaining! ;pp
5 minutes of sleep remaining! ;pp
4 minutes of sleep remaining! ;pp
3 minutes of sleep remaining! ;pp
2 minutes of sleep remaining! ;pp
1 minutes of sleep remaining! ;pp
0 minutes of sleep remaining! ;pp
Sleeping for 15 mins ...
14 minutes of sleep remaining! ;pp
13 minutes of sleep remaining! ;pp
12 minutes of sleep remaining! ;pp
11 minutes of sleep remaining! ;pp
10 minutes of sleep remaining! ;pp
9 minutes of sleep remaining! ;pp
8 minutes of sleep remaining! ;pp
7 minutes of sleep remaining! ;pp
6 minutes of sleep remaining! ;pp
5 minutes of sleep remaining! ;pp
4 minutes of sleep remaining! ;pp
3 minutes of sle

9 minutes of sleep remaining! ;pp
8 minutes of sleep remaining! ;pp
7 minutes of sleep remaining! ;pp
6 minutes of sleep remaining! ;pp
5 minutes of sleep remaining! ;pp
4 minutes of sleep remaining! ;pp
3 minutes of sleep remaining! ;pp
2 minutes of sleep remaining! ;pp
1 minutes of sleep remaining! ;pp
0 minutes of sleep remaining! ;pp
Sleeping for 15 mins ...
14 minutes of sleep remaining! ;pp
13 minutes of sleep remaining! ;pp
12 minutes of sleep remaining! ;pp
11 minutes of sleep remaining! ;pp
10 minutes of sleep remaining! ;pp
9 minutes of sleep remaining! ;pp
8 minutes of sleep remaining! ;pp
7 minutes of sleep remaining! ;pp
6 minutes of sleep remaining! ;pp
5 minutes of sleep remaining! ;pp
4 minutes of sleep remaining! ;pp
3 minutes of sleep remaining! ;pp
2 minutes of sleep remaining! ;pp
1 minutes of sleep remaining! ;pp
0 minutes of sleep remaining! ;pp
Sleeping for 15 mins ...
14 minutes of sleep remaining! ;pp
13 minutes of sleep remaining! ;pp
12 minutes of sleep remai

0 minutes of sleep remaining! ;pp
Sleeping for 15 mins ...
14 minutes of sleep remaining! ;pp
13 minutes of sleep remaining! ;pp
12 minutes of sleep remaining! ;pp
11 minutes of sleep remaining! ;pp
10 minutes of sleep remaining! ;pp
9 minutes of sleep remaining! ;pp
8 minutes of sleep remaining! ;pp
7 minutes of sleep remaining! ;pp
6 minutes of sleep remaining! ;pp
5 minutes of sleep remaining! ;pp
4 minutes of sleep remaining! ;pp
3 minutes of sleep remaining! ;pp
2 minutes of sleep remaining! ;pp
1 minutes of sleep remaining! ;pp
0 minutes of sleep remaining! ;pp
Done collecting 11 tweets for user @UNICEFDK
collecting tweets for: @noedhjaelp
Sleeping for 15 mins ...
14 minutes of sleep remaining! ;pp
13 minutes of sleep remaining! ;pp
12 minutes of sleep remaining! ;pp
11 minutes of sleep remaining! ;pp
10 minutes of sleep remaining! ;pp
9 minutes of sleep remaining! ;pp
8 minutes of sleep remaining! ;pp
7 minutes of sleep remaining! ;pp
6 minutes of sleep remaining! ;pp
5 minutes 

In [23]:
#save dict as json on computer
import json
def convert(date):
    if isinstance(date, datetime.datetime):
        return date.__str__()
    

with open('all_tweets_12.json', 'w') as outfile:
    json.dump(all_tweets_json, outfile, default=convert)

In [32]:
#inspecting tweet and information 
all_tweets['@PlanBornefonden'][0]

{'date': datetime.datetime(2021, 5, 14, 9, 3),
 'retweet': None,
 'tweet': '13-årige Larissa bor i Sahel-regionen, og var i 2019 tvunget til at flygte fra hendes landsby, fordi bevæbnede mænd angreb. \n\nI dag har en humanitær indsats givet hende muligheden for at starte på en skole i sikkerhed på trods af de militære uroligheder, der raser i hendes land. https://t.co/oDQKqVNZNt'}

In [12]:
#save dict to dataframe

actor1= []
tweet1 = []
date1 = []
retweet1 = []

#insepct dictionary
for user in all_tweets:
    print(user)
    for status in all_tweets[user]:
        actor1.append(user)
        tweet1.append(status['tweet'])
        date1.append(status['date'])
        retweet1.append(status['retweet'])
    
tweets_df_12 = pd.DataFrame()
(tweets_df_12['actor'], tweets_df_12['tweet'], 
 tweets_df_12['date'], tweets_df_12['retweet']) =  actor1, tweet1, date1, retweet1

@PlanBornefonden
@CARE_Danmark
@DRC_dk
@oxfamibis
@msf_dk
@ActionAidDK
@redbarnetdk
@danskrodekors
@SOS_Bornebyerne
@UNICEFDK
@noedhjaelp


In [13]:
print(tweets_df_12.shape) #number of tweets 
tweets_df_12.head()

(163, 4)


,actor,tweet,date,retweet
0,@PlanBornefonden,800 millioner piger og kvinder har menstruatio...,2021-05-26 06:26:00,None
1,@PlanBornefonden,Tak for en virkelig udbytterig paneldiskussion...,2021-05-25 11:33:28,None
2,@PlanBornefonden,Fondsdirektør @kimskibsted er på talerlisten o...,2021-05-25 10:31:18,RT @PDJF_dk:
3,@PlanBornefonden,@BLUETOWNwifi fortæller om deres arbejde med p...,2021-05-25 10:29:22,None
4,@PlanBornefonden,"Stephan Schönemann, vicedirektør i @DanishMFA ...",2021-05-25 09:51:25,None


In [14]:
#convert datetime (new column)
tweets_df_12['date_convert'] = pd.to_datetime(tweets_df_12['date']).dt.normalize()

In [15]:
tweets_df_12

,actor,tweet,date,retweet,date_convert
0,@PlanBornefonden,800 millioner piger og kvinder har menstruatio...,2021-05-26 06:26:00,None,2021-05-26
1,@PlanBornefonden,Tak for en virkelig udbytterig paneldiskussion...,2021-05-25 11:33:28,None,2021-05-25
2,@PlanBornefonden,Fondsdirektør @kimskibsted er på talerlisten o...,2021-05-25 10:31:18,RT @PDJF_dk:,2021-05-25
3,@PlanBornefonden,@BLUETOWNwifi fortæller om deres arbejde med p...,2021-05-25 10:29:22,None,2021-05-25
4,@PlanBornefonden,"Stephan Schönemann, vicedirektør i @DanishMFA ...",2021-05-25 09:51:25,None,2021-05-25
...,...,...,...,...,...
158,@noedhjaelp,Generalen går sjældent på gaden - men demo mod...,2021-05-19 15:23:16,None,2021-05-19
159,@noedhjaelp,Fælles udmelding fra danske NGO’er på Gaza: Ak...,2021-05-19 08:12:42,None,2021-05-19
160,@noedhjaelp,Vores chefrådgiver @sommerane følger tæt med i...,2021-05-18 12:46:44,None,2021-05-18
161,@noedhjaelp,»Jeg er 40 år. Jeg har boet i Gaza i hele mit ...,2021-05-18 06:19:59,None,2021-05-18


In [16]:
#save dataframe in excel file on computer 
tweets_df_12.to_excel("tweets_12_actors_15-26maj.xlsx") 

# Collecting tweets, second iteration

In [26]:
new_userlist = ['@lgbt_asylum', '@MissionEast', '@AVestegnen', '@CaritasDanmark']

In [17]:
#list of actors to scrape in second round 
userlist = ['@amnestydk', '@BornsVilkar', '@WFP_DK', '@danmissiondk', '@blaakorsdanmark',
           '@menneskeret', '@RTC_DK', '@DignityDK', '@UNDP_Danmark', '@globaltfokus',
           '@RefWelcome', '@DKIndsamling', '@Feministisk_DK', '@92_gruppen', '@ADRA_Danmark',
           ]

In [27]:
user_collected = [] # list of users already collected


In [28]:
userlist_to_collect = [user for user in new_userlist if user not in user_collected]

In [31]:
#for loop for scraping the new actors 

tweets = []
startDate = datetime.datetime(2020, 1, 1, 0, 0, 0)
endDate   = datetime.datetime(2021, 5, 28, 0, 0, 0)

all_tweets = {}

for user in userlist_to_collect:
    tweetCount = 0
    # Scraping status objects from the 'user' screen name
    user_tweets = []
    print("collecting tweets for:", user)
    c = tweepy.Cursor(api.user_timeline,
                      screen_name=user,
                      tweet_mode="extended" # to get the full text from a tweet.
                     ).items()
    while True:
        try:
            status = c.next()
            # check dato between start and end date, if so, save
            date = status.created_at
            if date < endDate and date >= startDate:

                #check for tweet or retweet
                retweet_match = re.search(r'^RT @\S+: ', status.full_text)
                if retweet_match:
                    tweet   = status.retweeted_status.full_text
                    retweet = retweet_match.group(0)
                else:
                    tweet   = status.full_text
                    retweet = None # use None to signal that this is not a retweet!
                user_tweets.append({"date": date, "tweet": tweet, "retweet": retweet})

                tweetCount += 1
        except tweepy.TweepError:
            sleep_15_mins()
            continue
        except StopIteration:
            break

    all_tweets[user] = user_tweets

    print('Done collecting {} tweets for user {}'.format(tweetCount, user))
        #user_collected.extend(user) # add the user to the list of user you have collected. 

    
#take either tweet or retweet extended 

collecting tweets for: @lgbt_asylum
Done collecting 107 tweets for user @lgbt_asylum
collecting tweets for: @MissionEast
Done collecting 101 tweets for user @MissionEast
collecting tweets for: @AVestegnen
Done collecting 294 tweets for user @AVestegnen
collecting tweets for: @CaritasDanmark
Done collecting 112 tweets for user @CaritasDanmark


In [32]:
#save dict to dataframe

actor1= []
tweet1 = []
date1 = []
retweet1 = []

#insepct dictionary
for user in all_tweets:
    print(user)
    for status in all_tweets[user]:
        actor1.append(user)
        tweet1.append(status['tweet'])
        date1.append(status['date'])
        retweet1.append(status['retweet'])

#Creating new dataframe for newly scraped actors         
        
tweets_new_actors = pd.DataFrame()
(tweets_new_actors['actor'], tweets_new_actors['tweet'], 
 tweets_new_actors['date'], tweets_new_actors['retweet']) =  actor1, tweet1, date1, retweet1

@lgbt_asylum
@MissionEast
@AVestegnen
@CaritasDanmark


In [33]:
tweets_new_actors

,actor,tweet,date,retweet
0,@lgbt_asylum,Vi ses til demo om lidt! Vi slår ring om syrer...,2021-05-19 13:25:56,None
1,@lgbt_asylum,Eid Mubarak til alle vores muslimske medlemmer...,2021-05-12 20:17:55,None
2,@lgbt_asylum,Vi præsenterer hermed det faglige notat om æn...,2021-05-12 20:13:13,RT @Beskytm_dk:
3,@lgbt_asylum,I dag eksamineres DK af FN's Menneskerettighed...,2021-05-06 06:55:15,RT @menneskeret:
4,@lgbt_asylum,Forholdene i Uganda går i den forkerte retning...,2021-05-04 07:44:30,None
...,...,...,...,...
609,@CaritasDanmark,Dolores Halpin-Bachmann har arbejdet for #Cari...,2020-02-07 10:49:47,None
610,@CaritasDanmark,Børn skal være børn - https://t.co/h5xNWh7a7U ...,2020-01-31 13:48:55,None
611,@CaritasDanmark,"Today, Caritas joined @ComeceEu for a @jpeurop...",2020-01-23 14:27:29,RT @CaritasEuropa:
612,@CaritasDanmark,”Ansvaret hviler i høj grad på jeres generatio...,2020-01-21 12:00:47,None


In [34]:
#convert datetime (new column)
tweets_new_actors['date_convert'] = pd.to_datetime(tweets_new_actors['date']).dt.normalize()

In [35]:
#save new dataframe in excel file on computer 
tweets_new_actors.to_excel("tweets_4_new_actors_.xlsx") 

# Lists of friends and followers

In [3]:
import configparser
from tweepy import API, Cursor, OAuthHandler, TweepError

In [83]:
print('Friends:', len(users_followers_friends['@PlanBornefonden']['friends']))
print('Followers:', len(users_followers_friends['@PlanBornefonden']['followers']))

Friends: 1469
Followers: 2004


In [84]:
#loop collecting ids

#create empty dict 
users_followers_friends = {}

for user in userlist_12:
    ids_friends   = []
    ids_followers = []

    print("Collecting followers for:", user) #collecting followers 
    #using twitter API 
    c = Cursor(api.followers_ids, screen_name = user).items()
    running = True
    while running:
        try: #try loop until API throws error, then break for 15 min 
            ids_followers.append(c.next())
        except tweepy.TweepError:
            sleep_15_mins()
            continue
        except StopIteration:
            running = False
    print(f'Done collecting {len(ids_followers)} followers')

    print("collecting friends for:", user) #collect friends 
    c = Cursor(api.friends_ids, screen_name=screen_name).items()
    running = True
    while running:
        try:
            ids_friends.append(c.next())
        except tweepy.TweepError:
            sleep_15_mins()
            continue
        except StopIteration:
            break
    #print every time a friends and followers list are collected         
    print(f'Done collecting {len(ids_friends)} friends')
    users_followers_friends[user] = {"followers": ids_followers, "friends": ids_friends}

Done collecting 2004 followers
collecting friends for: @PlanBornefonden
Done collecting 1469 friends
Done collecting 1728 followers
collecting friends for: @CARE_Danmark
Done collecting 1469 friends
Done collecting 6708 followers
collecting friends for: @DRC_dk
Done collecting 1469 friends
Done collecting 4700 followers
collecting friends for: @oxfamibis
Done collecting 1469 friends
Done collecting 5535 followers
collecting friends for: @msf_dk
Done collecting 1469 friends
Done collecting 4795 followers
collecting friends for: @ActionAidDK
Done collecting 1469 friends
Done collecting 7607 followers
collecting friends for: @redbarnetdk
Done collecting 1469 friends
Sleeping for 15 mins ...
14 minutes of sleep remaining! ;pp
13 minutes of sleep remaining! ;pp
12 minutes of sleep remaining! ;pp
11 minutes of sleep remaining! ;pp
10 minutes of sleep remaining! ;pp
9 minutes of sleep remaining! ;pp
8 minutes of sleep remaining! ;pp
7 minutes of sleep remaining! ;pp
6 minutes of sleep remaini

In [86]:
#loop collecting ids for the remaning actors 

for user in userlist:

    ids_friends   = []
    ids_followers = []

    print("Collecting followers for:", user)
    c = Cursor(api.followers_ids, screen_name = user).items()
    running = True
    while running:
        try:
            ids_followers.append(c.next())
        except tweepy.TweepError:
            sleep_15_mins()
            continue
        except StopIteration:
            running = False
    print(f'Done collecting {len(ids_followers)} followers')

    print("collecting friends for:", user)
    c = Cursor(api.friends_ids, screen_name=screen_name).items()
    running = True
    while running:
        try:
            ids_friends.append(c.next())
        except tweepy.TweepError:
            sleep_15_mins()
            continue
        except StopIteration:
            break
    print(f'Done collecting {len(ids_friends)} friends')
    users_followers_friends[user] = {"followers": ids_followers, "friends": ids_friends}

Done collecting 7248 followers
collecting friends for: @amnestydk
Done collecting 1469 friends
Done collecting 4595 followers
collecting friends for: @BornsVilkar
Done collecting 1469 friends
Done collecting 3650 followers
collecting friends for: @WFP_DK
Done collecting 1469 friends
Done collecting 975 followers
collecting friends for: @danmissiondk
Done collecting 1469 friends
Done collecting 1553 followers
collecting friends for: @blaakorsdanmark
Done collecting 1469 friends
Done collecting 5653 followers
collecting friends for: @menneskeret
Done collecting 1469 friends
Done collecting 425 followers
collecting friends for: @RTC_DK
Done collecting 1469 friends
Done collecting 2293 followers
collecting friends for: @DignityDK
Done collecting 1469 friends


Rate limit reached. Sleeping for: 885


Sleeping for 15 mins ...
14 minutes of sleep remaining! ;pp
13 minutes of sleep remaining! ;pp
12 minutes of sleep remaining! ;pp
11 minutes of sleep remaining! ;pp
10 minutes of sleep remaining! ;pp
9 minutes of sleep remaining! ;pp
8 minutes of sleep remaining! ;pp
7 minutes of sleep remaining! ;pp
6 minutes of sleep remaining! ;pp
5 minutes of sleep remaining! ;pp
4 minutes of sleep remaining! ;pp
3 minutes of sleep remaining! ;pp
2 minutes of sleep remaining! ;pp
1 minutes of sleep remaining! ;pp
0 minutes of sleep remaining! ;pp
Done collecting 4366 followers
collecting friends for: @UNDP_Danmark
Done collecting 1469 friends
Done collecting 2300 followers
collecting friends for: @globaltfokus
Done collecting 1469 friends
Done collecting 479 followers
collecting friends for: @RefWelcome
Done collecting 1469 friends
Done collecting 444 followers
collecting friends for: @DKIndsamling
Done collecting 1469 friends
Done collecting 1332 followers
collecting friends for: @Feministisk_DK
D

In [ ]:
actor1= []
tweet1 = []
date1 = []
retweet1 = []

#insepct dictionary
for user in all_tweets:
    print(user)
    for status in all_tweets[user]:
        actor1.append(user)
        tweet1.append(status['tweet'])
        date1.append(status['date'])
        retweet1.append(status['retweet'])

#Creating new dataframe for newly scraped actors         
        
tweets_new_actors = pd.DataFrame()
(tweets_new_actors['actor'], tweets_new_actors['tweet'], 
 tweets_new_actors['date'], tweets_new_actors['retweet']) =  actor1, tweet1, date1, retweet1

In [96]:


followers = []
user_list = []

#we want data this ways 
#{"Planbornefonden": {"followers": [1, 2, 3], "friends": [2, 3, 6]}, "redbarnet"}

#saving the data with the above structure 
for user in users_followers_friends:
    for user_id in users_followers_friends[user]["followers"]:
        user_list.append(user)
        followers.append(user_id)

followers_df = pd.DataFrame()
followers_df['actor'], followers_df['follower'] = user_list, followers

In [99]:
#SAVE DATA FOR FRIENDS

friends = []
user_list = []

#looping throuhg user's friends, saving them 
for user in users_followers_friends:
    for user_id in users_followers_friends[user]["friends"]:
        user_list.append(user)
        friends.append(user_id)

friends_df = pd.DataFrame()
friends_df['actor'], friends_df['friends'] = user_list, friends

In [105]:
#len of datasets 
print(friends_df.shape)
print(followers_df.shape)

#Save to CSV 
friends_df.to_csv('friends_df.csv')
followers_df.to_csv('followers_df.csv')

(38194, 2)
(91593, 2)


## From list of id's to usernames

In [11]:
#read data from computer 

friends_df = pd.read_csv('friends_df.csv')
friends_df #data only contains ids, and now we want the usernames 

,Unnamed: 0,actor,friends
0,0,@PlanBornefonden,4121845336
1,1,@PlanBornefonden,1381949996606558208
2,2,@PlanBornefonden,1053010495
3,3,@PlanBornefonden,867291066839556096
4,4,@PlanBornefonden,2725300877
...,...,...,...
38189,38189,@ADRA_Danmark,95868448
38190,38190,@ADRA_Danmark,14810076
38191,38191,@ADRA_Danmark,9261402
38192,38192,@ADRA_Danmark,58569858


In [ ]:
#reading the data for followers 
followers_df = pd.read_csv('followers_df.csv')
followers_df

In [ ]:
#Friend id to list 
friend_list = friends_df['friends'].tolist()
len(friend_list)

In [ ]:
#Followers id to list 
followers_list = followers_df['follower'].tolist()
len(followers_list)

In [ ]:
import traceback
import time


#Finding @usernames for id's (friends)
ids = friend_list
info = []
for i in range(0, len(ids), 100): #taking chunks of 100 
    try:
        chunk = ids[i:i+100]
        info.extend(api.lookup_users(user_ids=chunk))
    except:
        import traceback
        traceback.print_exc()
        print('Something went wrong, skipping...')
    time.sleep(10)


In [ ]:
#Saving names 
data = [x._json for x in info]
df = pd.DataFrame(data)
df = df[['id', 'name', 'screen_name']]
df.to_csv('friends_name.csv', index=False)

In [ ]:
#Finding @usernames for id's (followers)

import traceback
import time
ids = followers_list
info = []
for i in range(0, len(ids), 100):
    try:
        chunk = ids[i:i+100]
        info.extend(api.lookup_users(user_ids=chunk))
    except:
        import traceback
        traceback.print_exc()
        print('Something went wrong, skipping...')
    time.sleep(10)


In [ ]:
#saving followers names
data = [x._json for x in info]
df = pd.DataFrame(data)
df = df[['id', 'name', 'screen_name']]
df.to_csv('followers_name.csv', index=False)

In [ ]:
followers_name_df = pd.read_csv('followers_name.csv')
followers_name_df.head()

## Concatenate data frames with ID and ID names 

In [ ]:
#Making one friends dataset by merging the actors, friends ids and friends names 
friends_df1 = pd.concat([friends_df, friends_name_df], axis=1)
friends_df1.drop(['friends'], axis=1)
friends_df1.to_csv('friends_df1.csv', index=False)

In [ ]:
#Making one followers dataset by merging the actors, followers ids and followers names 
followers_df1 = pd.concat([followers_df, followers_name_df], axis=1)
followers_df1
followers_df1.to_csv('followers_df1.csv', index=False)